In [ ]:
import polars as pl
from record_consolidation.graphs import (
    unconsolidated_df_to_subgraphs,
)
from record_consolidation.subgraph_post_processing.specific_algs.deprecated__split_high_betweenness import (
    draw_graph,
)
import networkx as nx

from record_consolidation.subgraph_post_processing.specific_algs.partition_via_cluster_ensemble import (
    partition_subgraphs,
)

from record_consolidation.subgraph_post_processing.specific_algs.specific_partitioning_algs import (
    partition_via_louvain,
)

from functools import partial
from record_consolidation.graphs import atomize_records

from record_consolidation.df_consolidations import (
    normalize_subset,
)

/Users/mgaster/repos/record-consolidation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from typing import Iterable
from warnings import warn


def extract_specific_name_subgraph(
    connected_subgs: Iterable[nx.Graph], name: str
) -> nx.Graph:
    collected: list[nx.Graph] = []
    for subg in connected_subgs:
        for n in subg.nodes.data():
            if name.lower() in (n[0]).lower():
                collected.append(subg)
    collected = set(collected)
    if len(collected) == 0:
        raise ValueError(f"Could not find {name=} in connected_subgs.")
    if len(collected) > 1:
        alert_str = f"{len(collected)} subgraphs have {name=}.\n{[len(subg.nodes) for subg in collected]=}"
        warn(alert_str)
        return max(collected, key=lambda g: len(g.nodes))

        # raise ValueError(alert_str)
    return tuple(collected)[0]

In [3]:
# votes: pl.DataFrame = access_db_table("raw_output", "votes")
votes: pl.DataFrame = pl.read_parquet("raw_votes.parquet")
COMPANY_COLS = pl.col(["issuer_name", "cusip", "isin", "figi"])
COMPANY_COLS_L = ["issuer_name", "cusip", "isin", "figi"]
MEETING_COLS = pl.col(["issuer_name", "meeting_date", "cusip", "isin", "figi"])

In [4]:
# from record_consolidation.utils.polars_df import remove_string_nulls_and_uppercase


# raw_subgraphs = tuple(
#     unconsolidated_df_to_subgraphs(
#         votes.select(COMPANY_COLS),
#         connected_subgraphs_postprocessor=None,
#         pre_processing_fnc=remove_string_nulls_and_uppercase,
#     )
# )
# mattel_subg = extract_specific_name_subgraph(raw_subgraphs, "mattel")
# draw_graph(mattel_subg, 7)

In [5]:
# from record_consolidation.graphs import extract_consolidation_mapping_from_subgraphs
# from record_consolidation.utils.polars_df import remove_string_nulls_and_uppercase


# canonical_mapping = extract_consolidation_mapping_from_subgraphs(
#     unconsolidated_df_to_subgraphs(
#         votes.select(COMPANY_COLS),
#         connected_subgraphs_postprocessor=partition_companies_graph_where_necessary,
#         pre_processing_fnc=remove_string_nulls_and_uppercase,
#     )
# )

In [7]:
from record_consolidation.utils.polars_df import remove_string_nulls_and_uppercase


# normed5_intra = normalize_subset(
#     votes,
#     connected_subgraphs_postprocessor=partition_subgraphs,
#     cols_to_normalize=["issuer_name", "cusip", "isin", "figi"],
#     pre_processing_fnc_before_clustering=remove_string_nulls_and_uppercase,
#     consolidate_twice=True,
# )
normed5_NO_intra = normalize_subset(
    votes,
    connected_subgraphs_postprocessor=partition_subgraphs,
    id_colname="company_id",
    cols_to_normalize=["issuer_name", "cusip", "isin", "figi"],
    pre_processing_fnc_before_clustering=remove_string_nulls_and_uppercase,
    consolidate_twice=False,
)

Post-processing subgraphs.


7615it [00:03, 2520.76it/s]


Post-processing subgraphs.


7615it [00:01, 4041.71it/s]


shape: (6, 16)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ isin_issu ┆ cusip_iss ┆ figi_issu ┆ issuer_na ┆ … ┆ issuer_na ┆ cusip     ┆ isin      ┆ figi     │
│ er_name   ┆ uer_name  ┆ er_name   ┆ me_cusip  ┆   ┆ me        ┆ ---       ┆ ---       ┆ ---      │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ str       ┆ str       ┆ str      │
│ str       ┆ str       ┆ str       ┆ str       ┆   ┆ str       ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ US5949181 ┆ null      ┆ BBG000BPH ┆ null      ┆ … ┆ MICROSOFT ┆ 594918104 ┆ null      ┆ null     │
│ 045       ┆           ┆ 459       ┆           ┆   ┆ CORPORATI ┆           ┆           ┆          │
│           ┆           ┆           ┆           ┆   ┆ ON        ┆           ┆           ┆          │
│ US5949181 ┆ null      ┆ BBG000BPH ┆ null      ┆ … ┆ MICROSOFT ┆ 594918104 

In [19]:
normed5_NO_intra.group_by("company_id").agg(pl.col("cusip").n_unique())["cusip"].max()

1

In [7]:
jpmorgans = normed5_intra.filter(
    pl.col("issuer_name").str.contains("(?i)jpmorgan")
).sort(["accession", "order_recorded"], descending=False)
jpmorgans.write_csv("jpmorgans.csv")
jpmorgans["meeting_date"].value_counts(sort=True)

meeting_date,count
str,u32
"""2024-05-21""",2641
"""2024-05-22""",9
"""2024-05-20""",4
"""0024-05-20""",1
"""2024-05-24""",1
"""2134-07-25""",1


In [8]:
jpmorgans.write_csv("jpmorgans.csv")

In [9]:
jpmorgans["order_recorded"].value_counts(sort=True).head(30).to_pandas()

,order_recorded,count
0,0,787
1,1,132
2,2,106
3,3,94
4,4,92
5,5,92
6,6,91
7,7,91
8,8,91
9,9,91


In [10]:
for col in COMPANY_COLS_L:
    print(col)
    new_nulls = votes[col].is_not_null() & normed5_intra[col].is_null()
    print(votes.filter(new_nulls))

issuer_name
shape: (0, 24)
┌─────────────┬───────┬──────┬──────┬───┬───────────┬────────────┬─────────┬──────────────┐
│ issuer_name ┆ cusip ┆ isin ┆ figi ┆ … ┆ filer_cik ┆ filer_name ┆ fund_id ┆ fund_id_type │
│ ---         ┆ ---   ┆ ---  ┆ ---  ┆   ┆ ---       ┆ ---        ┆ ---     ┆ ---          │
│ str         ┆ str   ┆ str  ┆ str  ┆   ┆ str       ┆ str        ┆ str     ┆ str          │
╞═════════════╪═══════╪══════╪══════╪═══╪═══════════╪════════════╪═════════╪══════════════╡
└─────────────┴───────┴──────┴──────┴───┴───────────┴────────────┴─────────┴──────────────┘
cusip
shape: (0, 24)
┌─────────────┬───────┬──────┬──────┬───┬───────────┬────────────┬─────────┬──────────────┐
│ issuer_name ┆ cusip ┆ isin ┆ figi ┆ … ┆ filer_cik ┆ filer_name ┆ fund_id ┆ fund_id_type │
│ ---         ┆ ---   ┆ ---  ┆ ---  ┆   ┆ ---       ┆ ---        ┆ ---     ┆ ---          │
│ str         ┆ str   ┆ str  ┆ str  ┆   ┆ str       ┆ str        ┆ str     ┆ str          │
╞═════════════╪═══════╪══════╪══

In [11]:
# older

display(
    votes.select(COMPANY_COLS.is_null().sum())
    / normed5_intra.select(pl.all().is_null().sum())
)

issuer_name,cusip,isin,figi
f64,f64,f64,f64
NaN,4.743044,96.409019,1.513258


In [12]:
# prev

display(
    votes.select(COMPANY_COLS.is_null().sum())
    / normed5_intra.select(pl.all().is_null().sum())
)

issuer_name,cusip,isin,figi
f64,f64,f64,f64
NaN,4.743044,96.409019,1.513258


In [13]:
display(
    votes.select(COMPANY_COLS.is_null().sum())
    / normed5_intra.select(pl.all().is_null().sum())
)

issuer_name,cusip,isin,figi
f64,f64,f64,f64
NaN,4.743044,96.409019,1.513258


In [14]:
votes["issuer_name"].n_unique()

19277

In [15]:
normed5_intra["issuer_name"].value_counts(sort=True).head(30).to_series().to_list()

['MICROSOFT CORPORATION',
 'THE WALT DISNEY COMPANY',
 'AMAZON.COM, INC.',
 'ALPHABET INC.',
 'JPMORGAN CHASE & CO.',
 'THE PROCTER & GAMBLE COMPANY',
 'ORACLE CORPORATION',
 'CISCO SYSTEMS, INC.',
 'APPLE INC.',
 'JOHNSON & JOHNSON',
 'PEPSICO, INC.',
 'VISA INC.',
 'CHEVRON CORPORATION',
 'MASTERCARD INCORPORATED',
 'ABBVIE INC.',
 'BANK OF AMERICA CORPORATION',
 'MERCK & CO., INC.',
 'CHUBB LIMITED',
 'THE HOME DEPOT, INC.',
 'NVIDIA CORPORATION',
 'TE CONNECTIVITY LTD.',
 'EXXON MOBIL CORPORATION',
 'META PLATFORMS, INC.',
 'UNITEDHEALTH GROUP INCORPORATED',
 'VERIZON COMMUNICATIONS INC.',
 'THE COCA-COLA COMPANY',
 'SALESFORCE, INC.',
 "MCDONALD'S CORPORATION",
 'WELLS FARGO & COMPANY',
 'ADOBE INC.']

In [16]:
from record_consolidation.utils.polars_df import remove_string_nulls_and_uppercase


atomic = atomize_records(
    votes.select(COMPANY_COLS),
    partition_subgraphs,
    pre_processing_fnc=remove_string_nulls_and_uppercase,
)

Post-processing subgraphs.


7615it [00:02, 2621.26it/s]


In [17]:
atomic

isin,issuer_name,cusip,figi
str,str,str,str
null,null,"""74915M209""",null
null,"""1ST CAPITAL BANK""",null,null
null,"""?FIRST INTERSTATE BANCSYSTEM I…",null,null
null,"""A SPAC II ACQUISITION CORP.""","""G0543H125""",null
null,"""ABACUS GROUP""","""Q0015N260""",null
…,…,…,…
"""ZAE000302618""","""AVENG LTD""","""S0805F160""",null
"""ZAE000314084""","""OUTSURANCE GROUP LIMITED""","""S6815J126""",null
"""ZAE000322095""","""NAMPAK LTD""","""S5326R122""",null


In [18]:
names_comparison = pl.concat(
    [
        votes.select("issuer_name"),
        normed5_intra.select("issuer_name").rename(
            {"issuer_name": "normed_issuer_name"}
        ),
    ],
    how="horizontal",
)
names_comparison

issuer_name,normed_issuer_name
str,str
"""CANADIAN PACIFIC KANSAS CITY L…","""CANADIAN PACIFIC KANSAS CITY L…"
"""AAON, INC.""","""AAON, INC."""
"""AMAZON.COM, INC.""","""AMAZON.COM, INC."""
"""AMPHENOL CORPORATION""","""AMPHENOL CORPORATION"""
"""MICROSOFT CORPORATION""","""MICROSOFT CORPORATION"""
…,…
"""VERALTO CORPORATION""","""VERALTO CORPORATION"""
"""ANALOG DEVICES, INC.""","""ANALOG DEVICES, INC."""
"""EDITAS MEDICINE, INC.""","""EDITAS MEDICINE, INC."""


In [19]:
names_comparison.filter(pl.col("issuer_name").str.contains("(?i)conoco phillips"))

issuer_name,normed_issuer_name
str,str
"""Conoco Phillips""","""CONOCOPHILLIPS"""
"""Conoco Phillips""","""CONOCOPHILLIPS"""
"""Conoco Phillips""","""CONOCOPHILLIPS"""
"""Conoco Phillips""","""CONOCOPHILLIPS"""
"""Conoco Phillips""","""CONOCOPHILLIPS"""
…,…
"""CONOCO PHILLIPS""","""CONOCOPHILLIPS"""
"""CONOCO PHILLIPS""","""CONOCOPHILLIPS"""
"""CONOCO PHILLIPS""","""CONOCOPHILLIPS"""


In [20]:
names_comparison.filter(
    pl.col("normed_issuer_name") == pl.lit("MICROSOFT CORPORATION")
)["issuer_name"].value_counts(sort=True).with_columns(
    (pl.col("count") * 100 / pl.col("count").sum()).round(1).alias("%")
).to_pandas()

,issuer_name,count,%
0,MICROSOFT CORPORATION,2555,54.0
1,Microsoft Corporation,1795,37.9
2,Microsoft Corp,226,4.8
3,MICROSOFT\nCORPORATION,64,1.4
4,Microsoft Corp.,60,1.3
5,MICROSOFT CORP,13,0.3
6,Microsoft,6,0.1
7,MSFT,3,0.1
8,MICROSOFT,2,0.0
9,MICROSOFT CORPORATION COM,2,0.0


In [21]:
normed5_intra.filter(
    (pl.col("issuer_name") == pl.lit("MICROSOFT CORPORATION"))
    & (pl.col("cusip").is_null())
)

issuer_name,cusip,isin,figi,meeting_date,date_parsed_w_certainty,vote_description,other_vote_description,vote_source,management_recommendation,how_voted,shares_voted,total_shares_voted,shares_on_loan,vote_manager,vote_series,vote_other_info,accession,date_recorded,order_recorded,filer_cik,filer_name,fund_id,fund_id_type
str,str,str,str,str,i64,str,str,str,str,str,f64,f64,f64,null,str,str,str,str,i64,str,str,str,str


In [22]:
from record_consolidation import extract_normalized_atomic

ImportError: cannot import name 'extract_normalized_atomic' from 'record_consolidation' (/Users/mgaster/repos/record-consolidation/src/record_consolidation/__init__.py)

In [38]:
tom_records = pl.DataFrame(
    {
        "name": ["Tom"] * 10,
        "id": [12] * 7 + [None, 0, 12],
        "age": [50] * 5 + [51] * 5,
        "year_recorded": [2023] * 5 + [2024] * 5,
    }
)
print(tom_records)
normed_on_all = normalize_subset(
    tom_records.select(pl.col(["name", "id"])),
    cols_to_normalize="all",
    connected_subgraphs_postprocessor=partial(
        partition_subgraphs,
        verbose=False,
        verbose_within_partitioning_algs=False,
    ),
    pre_processing_fnc_before_clustering=None,
)
print(normed_on_all)
print(
    normalize_subset(
        tom_records,
        cols_to_normalize=["name", "id"],
        connected_subgraphs_postprocessor=partial(
            partition_subgraphs,
            verbose=False,
            verbose_within_partitioning_algs=False,
        ),
        pre_processing_fnc_before_clustering=None,
    )
)

print(
    extract_normalized_atomic(
        tom_records.select(["name", "id"]),
        pre_processing_fnc=None,
        connected_subgraphs_postprocessor=None,
    )
)

shape: (10, 4)
┌──────┬──────┬─────┬───────────────┐
│ name ┆ id   ┆ age ┆ year_recorded │
│ ---  ┆ ---  ┆ --- ┆ ---           │
│ str  ┆ i64  ┆ i64 ┆ i64           │
╞══════╪══════╪═════╪═══════════════╡
│ Tom  ┆ 12   ┆ 50  ┆ 2023          │
│ Tom  ┆ 12   ┆ 50  ┆ 2023          │
│ Tom  ┆ 12   ┆ 50  ┆ 2023          │
│ Tom  ┆ 12   ┆ 50  ┆ 2023          │
│ Tom  ┆ 12   ┆ 50  ┆ 2023          │
│ Tom  ┆ 12   ┆ 51  ┆ 2024          │
│ Tom  ┆ 12   ┆ 51  ┆ 2024          │
│ Tom  ┆ null ┆ 51  ┆ 2024          │
│ Tom  ┆ 0    ┆ 51  ┆ 2024          │
│ Tom  ┆ 12   ┆ 51  ┆ 2024          │
└──────┴──────┴─────┴───────────────┘
Post-processing subgraphs.


1it [00:00, 71.35it/s]


Post-processing subgraphs.


1it [00:00, 2087.76it/s]


shape: (10, 2)
┌──────┬─────┐
│ name ┆ id  │
│ ---  ┆ --- │
│ str  ┆ i64 │
╞══════╪═════╡
│ Tom  ┆ 12  │
│ Tom  ┆ 12  │
│ Tom  ┆ 12  │
│ Tom  ┆ 12  │
│ Tom  ┆ 12  │
│ Tom  ┆ 12  │
│ Tom  ┆ 12  │
│ Tom  ┆ 12  │
│ Tom  ┆ 12  │
│ Tom  ┆ 12  │
└──────┴─────┘
Post-processing subgraphs.


1it [00:00, 2109.81it/s]


Post-processing subgraphs.


1it [00:00, 1904.77it/s]

shape: (10, 4)
┌──────┬─────┬─────┬───────────────┐
│ name ┆ id  ┆ age ┆ year_recorded │
│ ---  ┆ --- ┆ --- ┆ ---           │
│ str  ┆ i64 ┆ i64 ┆ i64           │
╞══════╪═════╪═════╪═══════════════╡
│ Tom  ┆ 12  ┆ 50  ┆ 2023          │
│ Tom  ┆ 12  ┆ 50  ┆ 2023          │
│ Tom  ┆ 12  ┆ 50  ┆ 2023          │
│ Tom  ┆ 12  ┆ 50  ┆ 2023          │
│ Tom  ┆ 12  ┆ 50  ┆ 2023          │
│ Tom  ┆ 12  ┆ 51  ┆ 2024          │
│ Tom  ┆ 12  ┆ 51  ┆ 2024          │
│ Tom  ┆ 12  ┆ 51  ┆ 2024          │
│ Tom  ┆ 12  ┆ 51  ┆ 2024          │
│ Tom  ┆ 12  ┆ 51  ┆ 2024          │
└──────┴─────┴─────┴───────────────┘
shape: (1, 4)
┌─────┬───────────────┬──────┬─────┐
│ age ┆ year_recorded ┆ name ┆ id  │
│ --- ┆ ---           ┆ ---  ┆ --- │
│ i64 ┆ i64           ┆ str  ┆ i64 │
╞═════╪═══════════════╪══════╪═════╡
│ 50  ┆ 2023          ┆ Tom  ┆ 12  │
└─────┴───────────────┴──────┴─────┘


In [25]:
normed5_intra.filter(issuer_name="MICROSOFT CORPORATION").select(COMPANY_COLS).unique()

issuer_name,cusip,isin,figi
str,str,str,str
"""MICROSOFT CORPORATION""","""58155Q103""","""US58155Q1031""","""BBG000BLNPB7"""
"""MICROSOFT CORPORATION""","""594918104""","""US5949181045""","""BBG000BPH459"""


In [53]:
names_comparison.filter(
    pl.col("normed_issuer_name") == "MICROSOFT CORPORATION"
).group_by("issuer_name").agg(
    pl.col("normed_issuer_name").unique(),
    pl.len().alias("count"),
).with_columns(
    pl.col("normed_issuer_name").list[0],
    (pl.col("count") * 100 / pl.col("count").sum()).alias("%").round(1),
).sort(
    "count", descending=True
).to_pandas()

,issuer_name,normed_issuer_name,count,%
0,MICROSOFT CORPORATION,MICROSOFT CORPORATION,2555,54.0
1,Microsoft Corporation,MICROSOFT CORPORATION,1795,37.9
2,Microsoft Corp,MICROSOFT CORPORATION,226,4.8
3,MICROSOFT\nCORPORATION,MICROSOFT CORPORATION,64,1.4
4,Microsoft Corp.,MICROSOFT CORPORATION,60,1.3
5,MICROSOFT CORP,MICROSOFT CORPORATION,13,0.3
6,Microsoft,MICROSOFT CORPORATION,6,0.1
7,MSFT,MICROSOFT CORPORATION,3,0.1
8,MICROSOFTCORPORATION,MICROSOFT CORPORATION,2,0.0
9,MICROSOFT CORPORATION COM,MICROSOFT CORPORATION,2,0.0


In [29]:
normed5_NO_intra.filter(issuer_name="MICROSOFT CORPORATION").select(
    COMPANY_COLS
).to_struct().value_counts()

,count
struct[4],u32
"{""MICROSOFT CORPORATION"",null,""US5949181045"",""BBG000BPH459""}",30
"{""MICROSOFT CORPORATION"",""594918104"",""US5949181045"",""BBG000BPH459""}",4703


In [55]:
(30 / 4733) * 100

0.6338474540460596

In [59]:
lens = votes.group_by("issuer_name").agg(pl.len()).sort("len", descending=True)

In [66]:
lens.filter(
    pl.col("len").is_between(
        50,
        75,
    )
)

issuer_name,len
str,u32
"""The ODP Corporation""",75
"""London Stock Exchange Group pl…",75
"""Mohawk Industries, Inc.""",75
"""BeiGene, Ltd.""",75
"""SUNGROW POWER SUPPLY CO LTD""",75
…,…
"""JOHN BEAN TECHNOLOGIES CORPORA…",50
"""First American Financial Corpo…",50
"""Apogee Enterprises, Inc.""",50


In [67]:
for name in (
    # normed5["issuer_name"].value_counts(sort=True).head(30).to_series().to_list()
    [
        "odp",
        "beigene",
        "sungrow",
        "addus homecare",
        "microsoft",
        "disney",
        "amazon",
        "jpmorgan",
        "procter",
        "oracle",
        "apple",
        "visa",
        "exxon",
        "chevron",
        "meta platform",
        "air transport",
        "bank of america",
        "blackrock",
        "suncor",
        "conoco",
        "phillips 66",
    ]
):
    print("*" * 60, name.upper(), "*" * 60)
    t_unnormalized = (
        votes.select(COMPANY_COLS)
        .filter(pl.col("issuer_name").str.contains(f"(?i){name}"))
        .group_by("issuer_name")
        .agg(pl.len(), pl.all().unique())
        .with_columns((pl.col("len") * 100 / pl.col("len").sum()).alias("% of obs"))
        .sort("len", descending=True)
    )
    print(f"Un-Normalized (n={t_unnormalized["len"].sum()})")
    display(t_unnormalized)

    t_intra = (
        normed5_intra.select(COMPANY_COLS)
        .filter(pl.col("issuer_name").str.contains(f"(?i){name}"))
        .group_by("issuer_name")
        .agg(pl.len(), pl.all().unique())
        .with_columns((pl.col("len") * 100 / pl.col("len").sum()).alias("% of obs"))
        .sort("len", descending=True)
    )
    print(f"Normalized -- INTRA (n={t_intra["len"].sum()})")
    display(t_intra)

    t = (
        normed5_NO_intra.select(COMPANY_COLS)
        .filter(pl.col("issuer_name").str.contains(f"(?i){name}"))
        .group_by("issuer_name")
        .agg(pl.len(), pl.all().unique())
        .with_columns((pl.col("len") * 100 / pl.col("len").sum()).alias("% of obs"))
        .sort("len", descending=True)
    )
    print(f"Normalized -- NO INTRA (n={t["len"].sum()})")
    display(t)

    consolidation_fodder = (
        names_comparison.filter(
            pl.col("normed_issuer_name").str.contains(f"(?i){name}")
        )
        .select("issuer_name")
        .to_series()
        .value_counts(sort=True)
    )
    print("Consolidation Fodder:")
    display(consolidation_fodder.to_pandas())

************************************************************ ODP ************************************************************
Un-Normalized (n=132)


issuer_name,len,cusip,isin,figi,% of obs
str,u32,list[str],list[str],list[str],f64
"""The ODP Corporation""",75,"[""88337F105""]","[null, ""US88337F1057""]",[null],56.818182
"""THE ODP CORPORATION""",55,"[""88337F105"", null]","[""US88337F1057"", null]",[null],41.666667
"""THE ODP CORP.""",2,"[""88337F105""]",[null],[null],1.515152


Normalized -- INTRA (n=132)


issuer_name,len,cusip,isin,figi,% of obs
str,u32,list[str],list[str],list[str],f64
"""THE ODP CORPORATION""",132,"[""88337F105""]","[""US88337F1057""]",[null],100.0


Normalized -- NO INTRA (n=132)


...

issuer_name,len,cusip,isin,figi,% of obs
str,u32,list[str],list[str],list[str],f64
"""PHILLIPS 66""",722,"[""718546104""]","[""US7185461040""]","[""BBG00286S4N9""]",100.0


Normalized -- NO INTRA (n=722)


issuer_name,len,cusip,isin,figi,% of obs
str,u32,list[str],list[str],list[str],f64
"""PHILLIPS 66""",722,"[""718546104""]","[""US7185461040""]","[""BBG00286S4N9""]",100.0


Consolidation Fodder:


,issuer_name,count
0,PHILLIPS 66,420
1,Phillips 66,298
2,PHILLIPS 73,1
3,Phillips 68,1
4,Phillps 66,1
5,Phillips 66 (PSX),1


In [50]:
votes.filter(normed5["issuer_name"] != votes["issuer_name"]).select(COMPANY_COLS)

issuer_name,cusip,isin,figi
str,str,str,str
"""AMPHENOL CORPORATION""","""032095101""","""US0320951017""",null
"""THE SHERWIN-WILLIAMS COMPANY""","""824348106""","""US8243481061""",null
"""THE SHERWIN-WILLIAMS COMPANY""","""824348106""","""US8243481061""",null
"""AMPHENOL CORPORATION""","""032095101""","""US0320951017""",null
"""Toast, Inc.""","""888787108""",null,null
…,…,…,…
"""INTEL CORPORATION""","""458140100""",null,null
"""THE SHERWIN-WILLIAMS COMPANY""","""824348106""",null,null
"""THE KROGER CO.""","""501044101""",null,null


In [51]:
normed5.filter(normed5["issuer_name"] != votes["issuer_name"]).select(COMPANY_COLS)

issuer_name,cusip,isin,figi
str,str,str,str
"""Amphenol Corporation""","""032095101""","""US0320951017""",null
"""The Sherwin-Williams Company""","""824348106""","""US8243481061""","""BBG000BSXQV7"""
"""The Sherwin-Williams Company""","""824348106""","""US8243481061""","""BBG000BSXQV7"""
"""Amphenol Corporation""","""032095101""","""US0320951017""",null
"""TOAST, INC.""","""888787108""","""US8887871080""",null
…,…,…,…
"""Intel Corporation""","""458140100""","""US4581401001""","""BBG000C0G1D1"""
"""The Sherwin-Williams Company""","""824348106""","""US8243481061""","""BBG000BSXQV7"""
"""The Kroger Co.""","""501044101""","""US5010441013""","""BBG000BMY992"""


In [42]:
normed5["cusip"].n_unique()

7545

In [14]:
votes.select(COMPANY_COLS)

issuer_name,cusip,isin,figi
str,str,str,str
"""CANADIAN PACIFIC KANSAS CITY L…","""13646K108""","""CA13646K1084""",null
"""AAON, INC.""","""000360206""","""US0003602069""",null
"""AMAZON.COM, INC.""","""023135106""","""US0231351067""",null
"""AMPHENOL CORPORATION""","""032095101""","""US0320951017""",null
"""MICROSOFT CORPORATION""","""594918104""","""US5949181045""",null
…,…,…,…
"""VERALTO CORPORATION""","""92338C103""",null,null
"""ANALOG DEVICES, INC.""","""032654105""",null,null
"""EDITAS MEDICINE, INC.""","""28106W103""",null,null


In [6]:
normed5.select(pl.all().is_null().sum())

issuer_name,cusip,isin,figi
u32,u32,u32,u32
0,11703,8315,514694


In [ ]:
normed4 = normalize_subset4(
    votes,
    connected_subgraphs_postprocessor=partition_subgraphs,
    cols_to_normalize=["issuer_name", "cusip", "isin", "figi"],
)

In [ ]:
pp3 = partial(
    partition_subgraphs,
    # verbose=True,
)


normed3 = normalize_subset3(
    votes,
    connected_subgraphs_postprocessor=pp3,
    cols_to_normalize=["issuer_name", "cusip", "isin", "figi"],
)

In [ ]:
normed3

In [ ]:
normed3.select(COMPANY_COLS.is_null()).sum().sum_horizontal()

In [ ]:
votes.select(COMPANY_COLS.is_null().sum()).sum_horizontal().item()

## Check that removed nodes are repopulated

In [5]:
G = tuple(
    unconsolidated_df_to_subgraphs(
        votes.select(COMPANY_COLS), connected_subgraphs_postprocessor=None
    )
)

In [6]:
sb = extract_specific_name_subgraph(G, "STARBUCKS")

In [ ]:
len(sb.nodes)

In [ ]:
partitioned = partition_subgraphs(sb, verbose=True)

In [ ]:
print(len(partitioned.nodes))
draw_graph(partitioned, 5)

In [10]:
G = tuple(
    unconsolidated_df_to_subgraphs(
        votes.select(COMPANY_COLS), connected_subgraphs_postprocessor=None
    )
)

In [ ]:
list(G[0].nodes.data())[0]

In [ ]:
partition_via_louvain(G[4], verbose=True)
# draw_graph(G[4], 5)

In [ ]:
from record_consolidation.df_consolidations import normalize_subset

pp = partial(
    partition_subgraphs,
    # verbose=True,
)

atomic = atomize_records(
    votes.select(COMPANY_COLS), connected_subgraphs_postprocessor=pp
)
normed = normalize_subset(
    votes,
    connected_subgraphs_postprocessor=pp,
    cols_to_normalize=["issuer_name", "cusip", "isin", "figi"],
)

In [ ]:
votes.select(pl.all().is_null().sum())

In [ ]:
normed.select(
    pl.all().is_null().sum()
)  # .filter(pl.col("issuer_name") == pl.lit("AAON, INC."))

In [ ]:
atomic.select(pl.all().is_null().sum())

# ISSUE

## Problem
Nulls are being propagated in the 

## Diagnosis
(Best guess): When nodes are cut wholesale out of the graph - rather than even just having all edges cut - they don't end up in the canonical mapping, and so become null...

In [ ]:
removed_issuers = (
    votes.filter(normed["issuer_name"].is_null())
    .filter(pl.col("issuer_name").is_not_null())["issuer_name"]
    .unique()
    .to_list()
)
removed_issuers

In [ ]:
emerson = extract_specific_name_subgraph(G, "emerson electric co.")
draw_graph(emerson, 5)

In [ ]:
emerson_partitioned = partition_subgraphs(
    emerson, verbose=True, verbose_within_partitioning_algs=True
)

In [ ]:
for issuer in removed_issuers:
    try:
        draw_graph(extract_specific_name_subgraph(G, issuer), size=5)
    except:
        print(issuer)

In [ ]:
votes.select(COMPANY_COLS).filter(
    pl.col("issuer_name") == pl.lit("Northrup Grumman Corporation ")
)

In [ ]:
removed_issuers

In [ ]:
atomized_subset = atomize_records(
    votes.select(COMPANY_COLS), connected_subgraphs_postprocessor=pp
)

In [ ]:
s = "walmart"
atomized_subset.filter(pl.col("issuer_name").str.contains(f"(?i){s}"))

In [ ]:
atomized_subset.filter(pl.col("issuer_name").is_in(removed_issuers))

In [26]:
normed_comps = normed.select(COMPANY_COLS)

In [ ]:
normed_comps.filter(pl.col("issuer_name").is_null())

In [ ]:
normed3

In [ ]:
votes

In [ ]:
null_issuers = normed_comps["issuer_name"].is_null()


display(votes.select(COMPANY_COLS).filter(null_issuers).head())  # .sort(
#     pl.all()
# ).unique(maintain_order=True).to_pandas()
display(normed_comps.filter(null_issuers).head())

In [ ]:
atomized_subset.filter(cusip="291011104")

In [ ]:
normed.select(pl.all().is_null().sum())